In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
from functools import partial

In [9]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

envs Loaded
Pinged your deployment. You successfully connected to MongoDB!


In [10]:
sallaryMisDf = pd.read_excel("https://www.dropbox.com/scl/fi/zt2vtwhpz8ndblsxqdqx1/Salary_MIS.xlsx?rlkey=2uk6m7m9w90isv6zsynhhhpyv&st=gxumjns5&dl=1")

In [11]:
sallaryMisDf

Salary   GPA  MIS  Statistics
0        72  3.53    1           0
1        66  2.86    1           0
2        72  3.69    0           0
3        63  3.24    0           0
4        65  3.21    0           0
..      ...   ...  ...         ...
115      66  3.27    0           0
116      63  2.86    1           0
117      78  3.04    1           1
118      64  2.99    0           0
119      66  3.65    0           0

[120 rows x 4 columns]

In [12]:
sallaryMisDf.shape

(120, 4)

In [13]:
sallaryMisDf.describe()

Salary         GPA         MIS  Statistics
count  120.000000  120.000000  120.000000  120.000000
mean    69.875000    3.242750    0.316667    0.341667
std      6.594577    0.493834    0.467127    0.476257
min     53.000000    2.410000    0.000000    0.000000
25%     65.750000    2.805000    0.000000    0.000000
50%     70.000000    3.280000    0.000000    0.000000
75%     73.250000    3.692500    1.000000    1.000000
max     88.000000    3.980000    1.000000    1.000000

In [14]:
sallaryMisDf.shape

(120, 4)

In [15]:
sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])

const   GPA  MIS  Statistics
0      1.0  3.53    1           0
1      1.0  2.86    1           0
2      1.0  3.69    0           0
3      1.0  3.24    0           0
4      1.0  3.21    0           0
..     ...   ...  ...         ...
115    1.0  3.27    0           0
116    1.0  2.86    1           0
117    1.0  3.04    1           1
118    1.0  2.99    0           0
119    1.0  3.65    0           0

[120 rows x 4 columns]

In [16]:
salaryRegModel1 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])
)

In [17]:
salaryRegModel1Fit = salaryRegModel1.fit()

In [18]:
print(salaryRegModel1Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     150.3
Date:                Thu, 26 Jun 2025   Prob (F-statistic):           8.35e-40
Time:                        22:58:48   Log-Likelihood:                -300.92
No. Observations:                 120   AIC:                             609.8
Df Residuals:                     116   BIC:                             621.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.0072      1.860     23.662      0.0

In [19]:
sallaryMisDf

Salary   GPA  MIS  Statistics
0        72  3.53    1           0
1        66  2.86    1           0
2        72  3.69    0           0
3        63  3.24    0           0
4        65  3.21    0           0
..      ...   ...  ...         ...
115      66  3.27    0           0
116      63  2.86    1           0
117      78  3.04    1           1
118      64  2.99    0           0
119      66  3.65    0           0

[120 rows x 4 columns]

In [20]:
salaryRegModel1SampleData = {
    "GPA": [3.53, 3.04, 3.65],
    "MIS": [1, 1, 0],
    "Statistics": [0,1, 0]
}

In [21]:
def salaryRegModel1Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics']])
    return dfTransformer

In [22]:
def salaryRegModel1MainPredictor(self, transformedData):
    return self.predict(transformedData)

In [23]:
salaryRegModel1Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel1Fit)

In [24]:
salaryRegModel1Fit.transformer = salaryRegModel1Transformer

In [25]:
test1 = salaryRegModel1Fit.transformer(salaryRegModel1SampleData)
test1
# salaryRegModel1Fit.mainPredictor(test1)

const   GPA  MIS  Statistics
0    1.0  3.53    1           0
1    1.0  3.04    1           1
2    1.0  3.65    0           0

In [26]:
salaryRegModel1Fit.mainPredictor(test1)

0    73.992468
1    77.478273
2    68.180124
dtype: float64

In [27]:
salaryRegModel1FitMetadata = {
    "modelName":f"salaryRegModel1Fit",
    "version":"1.0.1",
    "params": salaryRegModel1Fit.params.to_dict(),
    "score": float(salaryRegModel1Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel1Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel1Fit.rsquared),
        "adj_r_squared": float(salaryRegModel1Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel1SampleData
    }
}
salaryRegModel1FitMetadata

{'modelName': 'salaryRegModel1Fit',
 'version': '1.0.1',
 'params': {'const': 44.00724634057483,
  'GPA': 6.622706150356178,
  'MIS': 6.607068893822078,
  'Statistics': 6.730931143358017},
 'score': 0.7953924097617404,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 3.2704010980898553e-46,
   'GPA': 3.077793077405891e-21,
   'MIS': 6.058766938187725e-20,
   'Statistics': 1.2379781357348497e-20},
  'r_squared': 0.7953924097617404,
  'adj_r_squared': 0.7901008341521303},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}}}

In [28]:
registerAJrjModel(salaryRegModel1Fit, salaryRegModel1FitMetadata)

✅ Uploaded encrypted ZIP to s3://273-g1/salaryRegModel1Fit__1.0.1.pkl.zip


{'_id': '685d7366250ee0d3933cb08b',
 'modelName': 'salaryRegModel1Fit',
 'version': '1.0.1',
 'params': {'const': 44.00724634057483,
  'GPA': 6.622706150356178,
  'MIS': 6.607068893822078,
  'Statistics': 6.730931143358017},
 'score': 0.7953924097617404,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 3.2704010980898553e-46,
   'GPA': 3.077793077405891e-21,
   'MIS': 6.058766938187725e-20,
   'Statistics': 1.2379781357348497e-20},
  'r_squared': 0.7953924097617404,
  'adj_r_squared': 0.7901008341521303},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}},
 'modelType': 'model',
 'keepLastOnly': False,
 'modelSizeBytes': 20657,
 'zippedModelSizeBytes': 7986,
 's3Url': '273-g1/salaryRegModel1Fit__1.0.1.pkl.zip',
 'createdAt': '2025-06-26T16:20:54.319Z',
 'updatedAt': '2025-06-26T16:20:54.319Z'}

In [29]:
sallaryMisDf

Salary   GPA  MIS  Statistics
0        72  3.53    1           0
1        66  2.86    1           0
2        72  3.69    0           0
3        63  3.24    0           0
4        65  3.21    0           0
..      ...   ...  ...         ...
115      66  3.27    0           0
116      63  2.86    1           0
117      78  3.04    1           1
118      64  2.99    0           0
119      66  3.65    0           0

[120 rows x 4 columns]

In [ ]:
sallaryMisDf["misXStatistics"] = sallaryMisDf["MIS"] * sallaryMisDf["Statistics"]
sallaryMisDf['misXStatistics1'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel2 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1"
      ]]
  )
)

In [ ]:
salaryRegModel2Fit = salaryRegModel2.fit()

In [ ]:
print(salaryRegModel2Fit.summary())

In [ ]:
salaryRegModel2SampleData = salaryRegModel1SampleData

In [ ]:
def salaryRegModel2Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['misXStatistics1'] = dfTransformer.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'misXStatistics1']])
    return dfTransformer

In [ ]:
salaryRegModel2Fit.transformer = salaryRegModel2Transformer

In [ ]:
salaryRegModel2Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel2Fit)

In [ ]:
test2 = salaryRegModel2Fit.transformer(salaryRegModel2SampleData)
test2

In [ ]:
salaryRegModel2Fit.mainPredictor(test2)

In [ ]:
salaryRegModel2FitMetadata = {
    "modelName":f"salaryRegModel2Fit",
    "version":"1.0.1",
    "params": salaryRegModel2Fit.params.to_dict(),
    "score": float(salaryRegModel2Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel2Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel2Fit.rsquared),
        "adj_r_squared": float(salaryRegModel2Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel2FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel2Fit, salaryRegModel2FitMetadata)

In [ ]:
sallaryMisDf['misXGpa'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['GPA'], axis=1)

sallaryMisDf


In [ ]:
salaryRegModel3 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
sallaryMisDf['statisticsXGpa'] = sallaryMisDf.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel4 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel4Fit = salaryRegModel4.fit()

In [ ]:
print(salaryRegModel4Fit.summary())

In [ ]:
salaryRegModel5 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1",
          "misXGpa",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel5Fit = salaryRegModel5.fit()

In [ ]:
print(salaryRegModel5Fit.summary())

In [ ]:
salaryRegModel6 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          # "MIS",
          "Statistics",
          "misXStatistics1",
          # "misXGpa",
          # "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel6Fit = salaryRegModel6.fit()

In [ ]:
print(salaryRegModel6Fit.summary())

## AFTER AREFULLY LOOKUP WE RELIZED  salaryRegModel4Fit SO LETS KEEP IT AS THE BEST

In [ ]:
def salaryRegModel4Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['statisticsXGpa'] = dfTransformer.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'statisticsXGpa']])
    return dfTransformer

In [ ]:
salaryRegModel4Fit.transformer = salaryRegModel4Transformer

In [ ]:
salaryRegModel4Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel4Fit)

In [ ]:
test4 = salaryRegModel4Fit.transformer(salaryRegModel2SampleData)
test4

In [ ]:
salaryRegModel4Fit.mainPredictor(test4)

In [ ]:
salaryRegModel4FitMetadata = {
    "modelName":f"salaryRegModel4Fit",
    "version":"1.0.1",
    "params": salaryRegModel4Fit.params.to_dict(),
    "score": float(salaryRegModel4Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel4Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel4Fit.rsquared),
        "adj_r_squared": float(salaryRegModel4Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel4FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel4Fit, salaryRegModel4FitMetadata)

In [ ]:
sallaryMisDf
